In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:

# Set the path to the directory containing the Bollywood actors dataset
dataset_path = "LFW\\lfw-deepfunneled"

# Function to load and preprocess images
def load_and_preprocess_image(actor_path, image_name):
    actor_name = os.path.basename(os.path.dirname(actor_path))
    image_path = os.path.join(dataset_path, actor_path, image_name)
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

# Function to create a more complex CNN model
def create_model():
    model = models.Sequential()

    # Convolutional layers with batch normalization and max pooling
    model.add(layers.Conv2D(32, (3, 3), activation='leaky_relu', input_shape=(224, 224, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='leaky_relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='leaky_relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='leaky_relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and fully connected layers with dropout
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='leaky_relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Load the pre-trained model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to load and preprocess training data
def load_and_preprocess_data(batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    return train_generator, validation_generator

# Load and preprocess training data
batch_size = 64
train_generator, validation_generator = load_and_preprocess_data(batch_size)

# Train the model
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Save the trained model
model.save("D:\\Coding\\trained_model.h5")

# Plot training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

plot_training_history(history)

In [ ]:
model.summary()

In [ ]:
def verify_face(user_actor, target_actor, user_image_name, target_image_name):
    user_img = load_and_preprocess_image(user_actor, user_image_name)
    target_img = load_and_preprocess_image(target_actor, target_image_name)

    # Predict embeddings for the images
    user_embedding = model.predict(user_img)
    target_embedding = model.predict(target_img)

    # Calculate Euclidean distance between embeddings
    distance = np.linalg.norm(user_embedding - target_embedding)

    # Set a threshold for verification
    threshold = 0.7  # You may need to tune this threshold based on your model and dataset

    # Perform verification
    if distance < threshold:
        return True  # Faces match
    else:
        return False  # Faces do not match

# Example usage
user_actor = "LFW\\lfw-deepfunneled\\Lionel_Messi\\"
target_actor = "LFW\\lfw-deepfunneled\\Lionel_Messi\\"
user_image_name = "Lionel_Messi_1.jpg"
target_image_name = "Lionel_Messi_2.jpg"

result = verify_face(user_actor, target_actor, user_image_name, target_image_name)

if result:
    print("Verification Successful: It's the same person!")
else:
    print("Verification Failed: It's not the same person.")

In [1]:
from flask import Flask, request, jsonify
import face_recognition
impoort os

app = Flask(__name__)

image_dir_path = []

def stream_directory_contents(directory_path):
    img_dir_paths = []
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                if entry.name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = entry.path
                    image_dir_path.append(image_path)
            elif entry.is_dir():
                stream_directory_contents(entry.path)


# image_dir_path = []

# def stream_directory_contents(directory_path):
#     img_dir_paths = []
#     with os.scandir(directory_path) as entries:
#         for entry in entries:
#             if entry.is_file():
#                 # print(entry.path)
#                 # Process only image files (you might want to add more file extensions)
#                 if entry.name.lower().endswith(('.png', '.jpg', '.jpeg')):
#                     image_path = entry.path
#                     image_dir_path.append(image_path)
#                     break
#                     # print(img_dir_paths)
#             elif entry.is_dir():
#                 # Recursive call for subdirectories
#                 stream_directory_contents(entry.path)
encoded_faces_list = []

def face_encoding(image):
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings[0]

def check_existing_user(image, encoded_list):
    enc = face_encoding(image)[0]
    results = face_recognition.compare_faces(encoded_list, enc)
    for i in len(results):
        if results[i] == True:
            return i
    return -1

@app.route('/face_recognition', methods=['POST'])
def recognize_face():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    image = face_recognition.load_image_file(request.files['image'])
    user_index = check_existing_user(image, encoded_faces_list)

    if user_index != -1:
        return jsonify({'user_id': user_index, 'message': 'User recognized'})

    return jsonify({'message': 'User not recognized'})

if __name__ == '__main__':
    app.run(debug=True)